<img src="/Users/zhangli/Library/Application Support/typora-user-images/image-20230615172857911.png">

##### seq2seq模型架构分析

- seq2seq模型分成两部分，分别是encoder(编码器)和decoder(解码器)，编码器和解码器的内部实现都使用GRU模型，这里它要完成的是一个中文到英文的翻译 欢迎 来 北京 --> welcome to BeiJing.编码器首先处理中文输入"欢迎 来 北京",通过GRU模型获取每个时间步的输出张量，最后将它们拼接成一个中间语义张量c，接着解码器将使用这个中间语义张量c以及每一个时间步的隐层张量，逐个生成对的翻译语言

##### 基于GRU的seq2seq模型架构实现翻译的过程

- 第一步：导入必备工具包
- 第二步：对持久化文件中的数据进行处理，以满足模型训练需求
- 第三步：构建基于GRU的编码器和解码器
- 第四步：构建模型训练函数，并进行训练
- 第五步：构建模型评估函数，并进行测试以及Attention效果分析


In [1]:
from io import open
# 用于字符规范化
import unicodedata
# 用于正则表达式
import re
# 用于随机生成数据
import random
# 用于构建网络结构和函数的torch工具包
import torch
import torch.nn as nn
import torch.nn.functional as F
# torch中预定义的优化方法工具包
from torch import optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#### 对持久化文件中的数据进行处理，以满足模型训练的要求

In [2]:
print(device)

cpu


In [3]:
# 将指定语言中的词汇映射成数值
# 起始标志
SOS_token = 0
# 结束标志
EOS_token = 1

class Lang:
    def __init__(self, name):
        """初始化函数中参数name代表传入某种语言的名字"""
        # 将name传入类中
        self.name = name
        # 初始化词汇对应自然数值的字典
        self.word2index = {}